# 資料預處理

資料預處理是資料分析和機器學習的重要步驟。其目的是將原始資料轉換為適合模型分析和學習的格式。資料預處理通常需要做以下幾件事情：

1. **資料清理**

   資料清理是指去除資料中的錯誤、不一致和缺失值。常見的資料清理方法包括：

   - 檢查資料的格式和類型
   - 處理異常值
   - 處理缺失值
   - 統一資料的格式和類型

2. **資料整合**

   資料整合是指將來自不同來源的資料合併在一起。常見的資料整合方法包括：

   - 匹配資料的欄位
   - 合併資料的表
   - 處理資料的衝突

3. **資料轉換**

   資料轉換是指將資料轉換為適合模型分析和學習的格式。常見的資料轉換方法包括：

   - 特徵縮放
   - 特徵編碼
   - 特徵選擇


## 檢查資料的格式和類型

In [1]:
import pandas as pd

df = pd.DataFrame({'a': [1, 2, 3], 'b': [4.5, 6.7, 8.9], 'c': ['Hello', 'world', '!']})

print(df.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       3 non-null      int64  
 1   b       3 non-null      float64
 2   c       3 non-null      object 
dtypes: float64(1), int64(1), object(1)
memory usage: 200.0+ bytes
None


In [4]:
# df.info()
# df.describe()

## 異常值 outlier
1. 如何判定異常值
2. 如何處理異常值或者空值

### 方法1  標準差：
- 在統計學中，如果一個數據分佈近似正態分佈，那麼大約68%的數據值在平均值的一個標準差內，約95%在兩個標準差內，約99.7%在三個標準差內。

In [37]:
import pandas as pd

iris = pd.read_csv('dataset/iris.csv', index_col = 0)
features = iris.iloc[:,:-1]

# features
# # 計算每個特徵的平均值和標準差
means = features.mean()
stds = features.std()
# 判定異常值
for i in features.index:
    for col in features.columns:
#         print(features.loc[i, col], means[col], stds[col])
        if abs(features.loc[i, col] - means[col]) > (3 * stds[col]):
            print(i, col)
            print(f'{col} 異常值：{iris.loc[i, col]}')

# # print(features)
# # for i in features.index:
# #     for col in features.columns:
# #         features = features[(features.loc[:, col] - means[col]).abs() < (3 * stds[col])]

# # print(features)

16 SepalWidthCm
SepalWidthCm 異常值：4.4


## 盒鬚圖
![](https://media.finebi.com/strapi/_06b36597d6.png)
- [盒鬚圖怎麼看](https://intl.finebi.com/zh-TW/blog/boxplot)

In [42]:
import pandas as pd

iris = pd.read_csv('dataset/iris.csv', index_col = 0)
iris = iris.iloc[:,:-1]

# 計算每個特徵的四分位數
q1 = iris.quantile(0.25)
q3 = iris.quantile(0.75)

# 判定異常值
for i in iris.index:
    for col in iris.columns:
        IQR = q3[col] - q1[col]
        if iris.loc[i, col] < q1[col] - 1.5 * IQR or iris.loc[i, col] > q3[col] + 1.5 * IQR:
            print(f'index:{i}, column:{col} 異常值：{iris.loc[i, col]}')


index:16, column:SepalWidthCm 異常值：4.4
index:33, column:SepalWidthCm 異常值：4.1
index:34, column:SepalWidthCm 異常值：4.2
index:61, column:SepalWidthCm 異常值：2.0


## DBSCAN 判定異常值的方法

- DBSCAN 判定異常值的方法是基於密度聚類來判定異常值。DBSCAN 會根據資料點之間的距離將資料點聚類在一起。密度較低的區域通常被認為是異常值。



In [3]:
import pandas as pd
from sklearn.cluster import DBSCAN

iris = pd.read_csv('dataset/iris.csv')
iris = iris.iloc[:,:-1]
# 訓練 DBSCAN 模型
dbscan = DBSCAN(eps=2, min_samples=2)
labels = dbscan.fit_predict(iris)

# 判定異常值
for i in range(iris.shape[0]):
    if labels[i] == -1:
        print(f'index:{i},異常值：{iris.loc[i, :]}')


index:57,異常值：Id               58.0
SepalLengthCm     4.9
SepalWidthCm      2.4
PetalLengthCm     3.3
PetalWidthCm      1.0
Name: 57, dtype: float64
index:106,異常值：Id               107.0
SepalLengthCm      4.9
SepalWidthCm       2.5
PetalLengthCm      4.5
PetalWidthCm       1.7
Name: 106, dtype: float64
index:122,異常值：Id               123.0
SepalLengthCm      7.7
SepalWidthCm       2.8
PetalLengthCm      6.7
PetalWidthCm       2.0
Name: 122, dtype: float64


## 異常值解釋
異常值（Outliers）是指在數據集中偏離大多數觀察值的一個或多個數據點。這些數據點通常與數據集中的其餘部分顯著不同，可能源於測量誤差、數據輸入錯誤、或**真實的異常情況**。異常值的存在可能會對統計分析和機器學習模型的性能產生重大影響，因為它們可以扭曲結果，使得模型對整體數據的解釋能力降低。因此，識別和處理異常值是資料分析中的一個關鍵步驟。

## 異常值處理方法

**刪除異常值**

刪除異常值是最簡單的處理異常值的方法。但是，這種方法可能會導致資料量的減少，從而影響資料分析結果的準確性。

**將異常值替換為平均值或中位數**

這種方法是使用統計方法來估算異常值。平均值是將所有非異常值的平均值用於替換異常值。中位數是將所有非異常值的眾數用於替換異常值。

- [延伸閱讀](https://allaboutdataanalysis.medium.com/%E6%AF%8F%E5%80%8B%E8%B3%87%E6%96%99%E7%A7%91%E5%AD%B8%E5%AE%B6%E6%87%89%E8%A9%B2%E7%9F%A5%E9%81%93%E7%9A%84%E4%BA%94%E7%A8%AE%E6%AA%A2%E6%B8%AC%E7%95%B0%E5%B8%B8%E5%80%BC%E7%9A%84%E6%96%B9%E6%B3%95-%E9%99%84python%E7%A8%8B%E5%BC%8F%E7%A2%BC-b9d790d16bf5)